# Event-Detecting Audio Recorder using Waggle

This notebook, based on previous work from the [SageEdu](https://github.com/ddiLab/SageEdu/tree/main/microphone) project, utilizes the PyWaggle library to make an audio recorder that can detect when there is sound to record.

When run, the notebook records for a predetermined amount of cycles, each of which is of a predetermined length. At the end of each cycle, the notebook briefly pauses recording so that it can go back and analyze the cycle in segments (frames). For each frame, the notebooks compares the maximum energy that occurs in it to a predetermined cutoff point. If it falls below the cutoff point, then the frame is discarded. The remaining frames are recombined, creating a number of audio events which are then saved to file. This way, the notebook only saves the parts of its recordings that actually contain sound information. After completing this step, the notebook begins the next cycle and resumes recording.

An analysis routine written for [SageEdu](https://github.com/ddiLab/SageEdu/blob/main/microphone/WaggleAudio.ipynb) can be run on events that have been recorded as well, printing basic information about the sound files and plotting their waveforms. Additionally, there is a routine to delete all previously recorded events.

## Setup

This sections contains our library imports and our function declarations.

In [ ]:
from waggle.data.audio import Microphone
from waggle.data.audio import AudioFolder
import matplotlib.pyplot as plt
import numpy as np
import soundfile
import time
import os

``record_events()`` is the primary recording routine. The user passes in a number of parameters to control aspects like the amount of cycles, the length of cycles, the cutoff energy, etc. As explained above, the notebook records for a given number of cycles, ending each cycle by looping through each frame it has just recorded and deciding whether to save it or discard it based on its max energy.

In [ ]:
def record_events(cycle, length, cutoff, sr, frame, path, overwrite):
    if overwrite:
        delete_events(path)
    
    microphone = Microphone()
    cycle_cnt = 1
    event_cnt_global = 0
    while True:
    
        if cycle_cnt-1 == cycle:
            break

        # record sample
        print(f"recording cycle {cycle_cnt}...")
        sample = microphone.record(length)
        event = False
        event_cnt = 0
        data = np.array([])
    
        # analyze sample by frames
        print(f"cycle {cycle_cnt} finished recording. analyzing for events...")
        for i in range(0, len(sample.data), frame): 
            # calculate max amplitude of current frame
            current_frame = sample.data[i:i+frame]
            max_current_frame = np.max(current_frame)
            if event: # if we're currently in an event,
                if max_current_frame > cutoff: # if event is still ongoing,
                    data = np.append(data, current_frame)
                else: # if event has ended,
                    event = False
                    event_cnt += 1
                    write_path = f"{path}/event_{cycle_cnt}_{event_cnt}.wav"
                    soundfile.write(str(write_path), np.append(data, current_frame), sr)
                    data = np.array([])
            else: # if we're not currently in an event
                if max_current_frame > cutoff: # if event has started,
                    event = True
                    data = np.append(data, current_frame)
                
        if event: # if cycle ends in the middle of an event, save it
            event_cnt += 1
            write_path = f"{path}/event_{cycle_cnt}_{event_cnt}.wav"
            soundfile.write(str(write_path), np.append(data, sample.data[-frame:]), sr)
            
        print(f"{event_cnt} events detected in cycle {cycle_cnt}\n")                
        
        cycle_cnt += 1
        event_cnt_global += event_cnt
    
    print(f"\n{cycle} cycles completed! {event_cnt_global} total events recorded.")

``analyze_events``iterates through a directory containing audio files and prints each file's sample rate, initial and final data points, and a waveform plot.

In [ ]:
def analyze_events(path):
    dataset = AudioFolder("audio_files") # desired directory
    x = 0
    for sample in dataset:
        link = str(dataset.files[x])
        print(f'samplerate: {sample.samplerate} \ntimestamp: {sample.timestamp} \ndata:\n{sample.data}\n\n')
        time2 = np.arange(0, len(sample.data) / sample.samplerate, 1/sample.samplerate)
    
        plt.figure(x)
        plt.title(link)
        plt.xlabel("Time [s]")
        plt.plot(time2, sample.data)
        plt.show()
        x += 1

``delete_events`` iterates through a directory and deletes all audio files of previously recorded events it contains.

In [ ]:
def delete_events(path):
    for x in os.listdir(path):
        if x[:6] == "event_":
            os.unlink(f"{path}/{x}")

## Running the Recorder

With our setup complete, we can now get to recording. The cell below declares all parameters used in the three functions established above:

- ``cycle`` - the amount of cycles to record for.
- ``length`` - the length, in seconds, of a recording cycle.
- ``cutoff`` - the energy point at which the recorder considers something to be an event. Calibrating this to the particular recording environment will take some experimentation. In my experience, 0.03 has worked as a good default. Using ``analyze_events`` can help give you an idea of what some of the recorded energies in your files look like.
- ``sr`` - the sample rate of the recordings, defaults to 48000.
- ``frame`` - the length of a frame in samples. To define it in seconds, multiply the amount of seconds by sr. Defaults to 1 second.
- ``path`` - the path of the directory to save audio files to. This path is also used when running the other two functions.
- ``overwrite`` - when set to true, the notebook will run ``delete_events`` before recording.

Once you've adjusted the parameters to your liking, run the cell, then run the cells below to run the functions.

In [ ]:
cycle = 3 # amount of recording cycles
length = 30 # length of recording cycles in seconds
cutoff = 0.03 # cutoff point for deciding whether something is an event (default = 0.05)
sr = 48000 # sample rate (default = 48000)
frame = sr # frame size in samples (default = sr (48000))
path = "audio_files"
overwrite = False

In [ ]:
record_events(cycle, length, cutoff, sr, frame, path, overwrite)

In [ ]:
analyze_events(path)

In [ ]:
delete_events(path)